<a href="https://colab.research.google.com/github/DeepManuPy/MultiLayer-Perceptrons/blob/master/Sonar.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import pandas as pd

seed=7
np.random.seed(seed)

In [0]:
df = pd.read_csv("sonar.csv")

dataset = df.values

X = dataset[:,:60].astype(float)
y = dataset[:,60]


In [0]:
from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()
enc_y = encoder.fit_transform(y)


In [0]:
import tensorflow as tf
from tensorflow.keras.layers import Dense,Dropout,BatchNormalization,Activation


def baseline_model():
  model = tf.keras.models.Sequential()
  model.add(Dense(60,input_dim=60,kernel_initializer='normal',activation='relu'))
  model.add(Dense(1,kernel_initializer='normal',activation='sigmoid'))
  
  model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])
  return model

def small_net():
  model = tf.keras.models.Sequential()
  model.add(Dense(30,input_dim=60,kernel_initializer='normal',activation='relu'))
  model.add(Dense(1,kernel_initializer='normal',activation='sigmoid'))
  model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])
  return model

def big_net():
  model = tf.keras.models.Sequential()
  model.add(Dense(60,input_dim=60,kernel_initializer='normal',activation='relu'))
  model.add(Dense(30,kernel_initializer='normal',activation='relu'))
  model.add(Dense(1,kernel_initializer='normal',activation='sigmoid'))
  
  model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])
  return model

In [0]:
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score,StratifiedKFold

model = KerasClassifier(baseline_model,epochs=200,batch_size=8,verbose=0)
kfold = StratifiedKFold(n_splits=10,shuffle=True,random_state=seed)

results = cross_val_score(model,X,enc_y,cv=kfold)

In [0]:
print("Baseline: %.2f%% (%.2f%%)"%(results.mean()*100,results.std()*100))

Baseline: 81.11% (6.29%)


#Standardizing input

In [0]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

sequence = []

sequence.append(('Standardize',StandardScaler()))
sequence.append(('mlp',KerasClassifier(baseline_model,epochs=200,batch_size=8,verbose=0)))

pipeline = Pipeline(sequence)
kfold = StratifiedKFold(n_splits=10,shuffle=True,random_state=seed)

results = cross_val_score(pipeline,X,enc_y,cv=kfold)

In [0]:
print("Baseline with Standardize: %.2f%% (%.2f%%)"%(results.mean()*100,results.std()*100))

Baseline with Standardize: 82.54% (4.65%)


# Evaluating a Smaller Network

  Reduced the number of neurons in the input layer by half. 
  So that, it forces the NN to extract important features from the given those of 60.

In [0]:
sequence = []

sequence.append(('Standardize',StandardScaler()))
sequence.append(('mlp',KerasClassifier(small_net,epochs=200,batch_size=8,verbose=0)))

pipeline = Pipeline(sequence)
kfold = StratifiedKFold(n_splits=10,shuffle=True,random_state=seed)

results = cross_val_score(pipeline,X,enc_y,cv=kfold)

In [0]:
print("Small network with Standardize: %.2f%% (%.2f%%)"%(results.mean()*100,results.std()*100))

Small network with Standardize: 83.52% (6.29%)


#Evaluating a Bigger Network


In [0]:
sequence = []

sequence.append(('Standardize',StandardScaler()))
sequence.append(('mlp',KerasClassifier(big_net,epochs=200,batch_size=8,verbose=0)))

pipeline = Pipeline(sequence)
kfold = StratifiedKFold(n_splits=10,shuffle=True,random_state=seed)

results = cross_val_score(pipeline,X,enc_y,cv=kfold)

In [0]:
print("Big network with Standardize: %.2f%% (%.2f%%)"%(results.mean()*100,results.std()*100))

Big network with Standardize: 84.04% (6.78%)
